## Trame imports

In [1]:
from trame.app import get_server, jupyter
from trame.widgets import vtk, trame, vuetify, html
from trame.ui.vuetify import SinglePageLayout

## VTK code

In [2]:
from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkRenderer,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkPolyDataMapper,
    vtkActor,
)
import vtkmodules.vtkRenderingOpenGL2  # noqa
from vtkmodules.vtkInteractionStyle import vtkInteractorStyleSwitch  # noqa

DEFAULT_RESOLUTION = 6

renderer = vtkRenderer()
renderWindow = vtkRenderWindow()
renderWindow.AddRenderer(renderer)

renderWindowInteractor = vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.GetInteractorStyle().SetCurrentStyleToTrackballCamera()

cone_source = vtkConeSource()
mapper = vtkPolyDataMapper()
actor = vtkActor()
mapper.SetInputConnection(cone_source.GetOutputPort())
actor.SetMapper(mapper)
renderer.AddActor(actor)
renderer.ResetCamera()
renderWindow.Render()

## Trame setup

In [3]:
server = get_server("jupyter-1")
state, ctrl = server.state, server.controller


def reset_resolution():
    state.resolution = 6


@state.change("resolution")
def update_resolution(resolution, **kwargs):
    cone_source.SetResolution(int(resolution))
    ctrl.view_update()


## UI setup

In [4]:
layout = SinglePageLayout(server)

with layout:
    # Validate client life cycle
    trame.LifeCycleMonitor(events=("['created']",))

    layout.icon.click = ctrl.reset_camera
    layout.title.set_text("Cone")
    layout.toolbar.dense = True

    # Toolbar
    with layout.toolbar as toolbar:
        vuetify.VSpacer()
        vuetify.VSlider(
            hide_details=True,
            v_model=("resolution", 6),
            max=60,
            min=3,
            step=1,
            style="max-width: 300px;",
        )
        vuetify.VSwitch(
            hide_details=True,
            v_model=("$vuetify.theme.dark",),
        )
        with vuetify.VBtn(icon=True, click=reset_resolution):
            vuetify.VIcon("mdi-undo")

    with layout.content:
        with vuetify.VContainer(fluid=True, classes="pa-0 fill-height"):
            view = vtk.VtkRemoteView(renderWindow)
            ctrl.view_update = view.update
            ctrl.reset_camera = view.reset_camera


## Run inside jupyter

In [5]:
jupyter.show(server)

## Another app

In [6]:
# -----------------------------------------------------------------------------
# Trame setup
# -----------------------------------------------------------------------------

s2 = get_server("jupyter-2")
state2, ctrl2 = s2.state, s2.controller


def reset_resolution_2():
    state2.resolution = 6

ctrl2.reset_resolution = reset_resolution_2

# -----------------------------------------------------------------------------
# UI setup
# -----------------------------------------------------------------------------

layout2 = SinglePageLayout(s2)

with layout2:
    # Validate client life cycle
    trame.LifeCycleMonitor(events=("['created']",))

    layout2.icon.click = ctrl2.reset_camera
    layout2.title.set_text("Cone")
    layout2.toolbar.dense = True

    # Toolbar
    with layout2.toolbar as toolbar:
        vuetify.VSpacer()
        vuetify.VSlider(
            hide_details=True,
            v_model=("resolution", 6),
            max=60,
            min=3,
            step=1,
            style="max-width: 300px;",
        )
        vuetify.VSwitch(
            hide_details=True,
            v_model=("$vuetify.theme.dark",),
        )
        with vuetify.VBtn(icon=True, click=ctrl2.reset_resolution):
            vuetify.VIcon("mdi-undo")

    with layout2.content:
        with vuetify.VContainer(fluid=True, classes="pa-0 fill-height"):
            with vtk.VtkView() as view:
                ctrl2.reset_camera = view.reset_camera
                with vtk.VtkGeometryRepresentation():
                    vtk.VtkAlgorithm(
                        vtk_class="vtkConeSource", state=("{ resolution }",)
                    )


In [7]:
jupyter.show(s2)

In [8]:
jupyter.show(server)
jupyter.show(s2)

In [9]:
jupyter.show(server)
jupyter.show(server)